# Example N27-2

Get an overview of levee charcateristics for levee trajectory 27-2.

Thanks to package `GeoProfile` that allows to visualise geotechnical data in Python.

In [1]:
import os
import pickle
import pandas as pd
import numpy as np

import datetime

import logging
logging.basicConfig(level=logging.DEBUG)
# when logging level = DEBUG, skip messages from matplotlib and PIL
for lib in ['rasterio', 'matplotlib', 'PIL', 'urllib3', 'choreographer', 'kaleido']:
    logging.getLogger(lib).setLevel(logging.INFO)


In [2]:
import hydropandas as hpd
hpd.__version__

'0.18.0'

In [3]:
import geoprofile
geoprofile.__version__

'0.4.0'

In [4]:
# TODO: De functies van `geolookup` staan nu allemaal in een .py bestand. Dit nog Python-ic maken. Met een init.py en verschillende sub besatanden


In [5]:
import sys
sys.path.append(r'..\..\geolookup')

import functions as gl

# Gegevens openen

In [6]:
DATA_PATH = r'N27-2_data\\'
PLOT_PATH = r'N27-2_plots\\'

## CPTs

Downloaded from Broloket and classified via `CPTcore` using Robertson classification.

In [7]:
# TODO: change method to plot CPTs in section perpendicular to levee. Now data is projected using Waterschap Scheldestromen method to
# project CPTs relative to reference line (x=0). Therefore, preprocessing is done outside this repo.

In [8]:
region = 'Os'
dp_min = 725
dp_max = 1080
cpt_classification = 'Robertson'

fn = DATA_PATH + f'geoprofile_columns_robertson_{region}_dp{dp_min}-{dp_max}.pkl'
logging.info(f"File last modified: {datetime.datetime.fromtimestamp(os.path.getmtime(fn)) - datetime.datetime.now()}")
with open(fn, "rb") as f:
    geoprofile_cols = pickle.load(f)
gl.describe_columns(geoprofile_cols)

INFO:root:File last modified: -5 days, 2:18:08.754255
INFO:root:Number of columns: 1248
INFO:root:Dike pole range: dp726.6 - dp1081.5, total 759 unique dike poles.
INFO:root:Data X: centered at 64380 with span 13 km, Y: centered at 397930 with span 11 km


## Hectometer indicator

In [9]:
df_hm = pd.read_csv(DATA_PATH + f'table_dijkpalen_{region}_dp{dp_min}-{dp_max}.csv')
df_hm.dp.describe()

count     711.000000
mean      902.500000
std       102.696154
min       725.000000
25%       813.750000
50%       902.500000
75%       991.250000
max      1080.000000
Name: dp, dtype: float64

## Schematised levee profiles

In [10]:
df_profiles = pd.read_csv(DATA_PATH + f'table_profielen_{region}_dp{dp_min}-{dp_max}.csv')
df_profiles.head()

,region,dp,ymv.bui,ysloot.1d,xmv.bin,xsloot.1a,xsloot.1c,xsloot.1d,xsloot.1b,xweg.1,...,yteen.1,yberm.1a,yberm.1b,ykruin.1,ykruin.2,yberm.2a,yberm.2b,yteen.2,yweg.2,ysloot.2
0,Os,725.0,-0.253,-0.5,-91.13,-52.13,-50.79,-50.39,-49.13,-25.13,...,1.71,6.26,6.39,6.39,6.22,4.63,4.33,0.03,0.773,-0.071
1,Os,726.0,-0.583,0.0,-200.88,-97.88,-95.77,-92.82,-90.88,-18.88,...,2.45,6.43,6.43,6.43,6.41,4.58,4.28,0.05,-0.700,-0.640
2,Os,727.0,-0.928,0.5,-51.22,-23.22,-21.81,-20.57,-19.22,-19.22,...,2.08,6.51,6.51,6.51,6.43,4.53,4.27,-0.74,-0.776,-0.776
3,Os,728.0,-0.926,0.5,-201.00,-24.00,-22.64,-22.60,-21.00,-21.00,...,1.86,6.45,6.45,6.45,6.37,4.44,4.23,0.26,-0.130,-0.929
4,Os,729.0,-1.034,0.5,-201.00,-23.62,-22.34,-20.45,-19.00,-19.00,...,1.50,6.46,6.46,6.46,6.42,4.48,4.25,-0.76,-1.046,-0.936


## Section profiles

Distance along trajectory where drawing (section profile) is available.

In [11]:
df_section = pd.read_csv(DATA_PATH + f'table_deltaversterking_{region}_dp{dp_min}-{dp_max}.csv')
df_section['description'] = 'Deltaversterking'
df_section.dsn_file_tif = DATA_PATH + df_section.dsn_file_tif 
df_section.head()

,region,dp,dsn_file_tif,description
0,Os,725.5,N27-2_data\\deltaversterking\os0725-0778\doors...,Deltaversterking
1,Os,730.5,N27-2_data\\deltaversterking\os0725-0778\doors...,Deltaversterking
2,Os,734.5,N27-2_data\\deltaversterking\os0725-0778\doors...,Deltaversterking
3,Os,736.5,N27-2_data\\deltaversterking\os0725-0778\doors...,Deltaversterking
4,Os,744.9,N27-2_data\\deltaversterking\os0725-0778\doors...,Deltaversterking


## Groundwater monitoring and time series
Using `HydroPandas`

In [12]:
#TODO: functie maken die ouderom netjes aangeeft, als > dag, in dagen, anders in uren, anders in minuten
fn = DATA_PATH + f'oc_gwl_{region}_dp{dp_min}-{dp_max}.pkl'
oc_gwl = hpd.read_pickle(fn)
logging.info(f"File last modified: {(datetime.datetime.fromtimestamp(os.path.getmtime(fn)) - datetime.datetime.now()).total_seconds() / 60:.2f} minutes")
gl.describe_oc(oc_gwl, expected_obs_interval=10)


INFO:root:File last modified: -7106.03 minutes
INFO:root:Obs Collection has 85 monitoring wells, on 66 locations. First observation 2007-08-29, last observation 2025-11-22
INFO:root:Obs Collection has on average 1.1 years of observations per monitoring well, median 1.3 years. Assuming observations interval each 10 minutes.
INFO:root:Data X: centered at 64220 with span 13 km, Y: centered at 397920 with span 10 km
INFO:root:Index is unique.


In [13]:
# not all standpipes are in Broloket at the moment (work in progess, I know)
# standpipes that are in Broloket are downloaded, distances along and perpendicular to levee added
df_gmws = pd.read_csv(DATA_PATH + 'gmws_met_dp_en_afstanden.csv', index_col=0)
print("Is index unique?", df_gmws.index.is_unique)
df_gmws.head()


Is index unique? True


,broId,tubeNumber,constructionStandard,coordinateTransformation,corrected,deliveredLocation,deliveryAccountableParty,deliveryContext,deregistered,groundLevelPosition,...,screenBottomPosition,plainTubePartLength,sedimentSumpLength,x,y,geometry,ref_distance,dp,distance_to_ref,region
bro_identifier,,,,,,,,,,,,,,,,,,,,,
GMW000000064173-1,GMW000000064173,1,NEN5104,RDNAPTRANS2018,nee,POINT (66087.627 402728.068),51640813,publiekeTaak,nee,0.485,...,-1.315,0.70,NaN,66087.627,402728.068,POINT (66087.627 402728.068),78437.0,784.37,-30.560607,Os
GMW000000064174-1,GMW000000064174,1,NEN5104,RDNAPTRANS2018,nee,POINT (66063.802 402740.143),51640813,publiekeTaak,nee,0.552,...,-1.448,0.88,NaN,66063.802,402740.143,POINT (66063.802 402740.143),78443.0,784.43,-56.800483,Os
GMW000000064635-1,GMW000000064635,1,NEN5104,RDNAPTRANS2018,nee,POINT (70954.224 392370.279),51640813,publiekeTaak,nee,3.112,...,-0.888,2.95,1.0,70954.224,392370.279,POINT (70954.224 392370.279),108150.0,1081.50,16.003035,Os
GMW000000064641-1,GMW000000064641,1,NEN5104,RDNAPTRANS2018,nee,POINT (70959.858 392366.996),51640813,publiekeTaak,nee,4.737,...,-2.263,5.95,1.0,70959.858,392366.996,POINT (70959.858 392366.996),108151.0,1081.51,9.511578,Os
GMW000000082197-1,GMW000000082197,1,NEN5104,RDNAPTRANS2018,nee,POINT (51425.398 406745.471),51640813,WW,nee,9.270,...,-14.731,22.87,1.0,51425.398,406745.471,POINT (51425.398 406745.471),18001.0,180.01,-1.756535,Os


# Section along the trajectory of the levee

Using the standard `profile` function of `GeoProfile`. It is called within a specific `GeoLookup` function, that creates the path.
Afterwards levee charecteristics are added to the plot. That uses the `dp` (hectometer) property of these data sets.

Section opens in a browser window. Map is included below.

In [14]:
if 0:
    region = 'Os'
    dp_list = [818, 821]
    dp_list = np.arange(818, 821+1)
    #dp_list = [930, 936]
    fig, profile = gl.plot_geoprofile(
        geoprofile_cols, 
        df_hm, 
        dp_list,
        oc_gwl=oc_gwl, 
        buffer=60, 
        projectname=f"Along levee {region} {min(dp_list)}-{max(dp_list)} {cpt_classification}",
        title_suffix='',
        plot_path=PLOT_PATH,)

In [15]:
# TODO: plotting of standpipes via oc_gwl is not working visible in maps. Annotation is plotted, legend as well. Standpipe itself not visible.

# Doorsnede maken bij dijkpaal

Een doorsnede haaks op de dijk, ter hoogte van een dijkpaal. Weergegeven wordt het beschikbare grondonderzoek, de dichtstbijzijnde versterkingstekening van de Deltaversterking, het actuele maaiveld profiel en de peilbuizen.

Informatie van meerdere dijkpalen wordt weergegeven wanneer de `delta_dp` variabele wort gebuikt. Informatie van lagere dijkpalen krijgt een rode tint, informatie van hogere dijkpalen een blauwe.

Grondonderzoek wordt geprojecteerd ten opzichte van de referentielijn. Het wordt nooit bovenopelkaargeplot. Dan krijgt het een offset. Die offset is weergegeven tussen de haakjes in het label. Grondonderzoek van de opgevraagde raai wordt als eerste geplot, en staat dus vrijwel altijd op de juiste plek.

In [16]:
dp_center = 820
delta_dp = 1

df_gmws['position'] = 'broloket'
df_gmws = df_gmws.rename(columns={'screenTopPosition': 'screen_top',
                        'screenBottomPosition': 'screen_bottom',})

df_plot = df_gmws.loc[df_gmws.dp.between(
            dp_center - delta_dp, dp_center + delta_dp) & (df_gmws.region == region)].copy()

# Extract the number of zeros after the first three characters (e.g., after 'GMW')
min_padding_zeros = (
    df_plot.index.str[3:]
    .str.extract(r'^(0+)')[0]
    .str.len()
    .min()
)
print("Minimum number of padding zeros in index:", min_padding_zeros)

df_plot['label'] = df_plot.index.str.replace(
    r'^(GMW)(0{%d})' % min_padding_zeros, r'\1', regex=True
)


gl.create_figure_for_dp(
    dp_center, 
    df_section, 
    geoprofile_cols, 
    df_profiles,
    oc_gwl,
    df_gmws=df_plot,
    delta_dp=delta_dp,
    plot_gwl=True,
    plot_path=PLOT_PATH,
    ylims_upper=[-4, 7]
    )

DEBUG:root:Closest dp with Deltaversterking to 820 is 818.1, fn: es\dsn10_flipped.tif


Minimum number of padding zeros in index: 7


DEBUG:root:Adding Deltaversterking background from 9\doorsnedes\dsn10_flipped.tif to figure at row 1, col 1.
DEBUG:root:Plotting column dp 820.0, distance=-2.763735125639539, plotted=[np.float64(-3.0)]
DEBUG:root:Plotting column dp 820.0, distance=-14.7556743357624, plotted=[np.float64(-3.0), np.float64(-15.0)]
DEBUG:root:Plotting column dp 819.5, distance=-15.441061606105023, plotted=[np.float64(-3.0), np.float64(-15.0), np.float64(-16.0)]
DEBUG:root:Plotting column dp 820.5, distance=-17.26704174450618, plotted=[np.float64(-3.0), np.float64(-15.0), np.float64(-16.0), np.float64(-17.0)]
DEBUG:root:Plotting column dp 819.0, distance=-2.161589776560799, plotted=[np.float64(-3.0), np.float64(-15.0), np.float64(-16.0), np.float64(-17.0), np.float64(-2.0)]
DEBUG:root:Plotting column dp 819.0, distance=-19.641947163776827, plotted=[np.float64(-3.0), np.float64(-15.0), np.float64(-16.0), np.float64(-17.0), np.float64(-2.0), np.float64(-20.0)]
DEBUG:root:Plotting column dp 821.0, distance=-19

(Figure({
     'data': [{'fill': 'toself',
               'fillcolor': '#008000',
               'fillpattern': {'shape': '/'},
               'hovertemplate': '%{text}',
               'legendgroup': np.str_('klei'),
               'line': {'color': '#008000'},
               'mode': 'lines',
               'name': np.str_('klei'),
               'showlegend': True,
               'text': ('Name: CPT000000227712<br>Soil ' ... 'layer: 5.14<br>Thickness: 0.52'),
               'type': 'scatter',
               'x': [-3.4, -3.4, -2.4, -2.4, -3.4],
               'xaxis': 'x',
               'y': [5.657, 5.1370000000000005, 5.1370000000000005, 5.657, 5.657],
               'yaxis': 'y'},
              {'fill': 'toself',
               'fillcolor': '#008000',
               'fillpattern': {'shape': '/'},
               'hovertemplate': '%{text}',
               'legendgroup': np.str_('klei'),
               'line': {'color': '#008000'},
               'mode': 'lines',
               'name'

In [17]:
df_plot

,broId,tubeNumber,constructionStandard,coordinateTransformation,corrected,deliveredLocation,deliveryAccountableParty,deliveryContext,deregistered,groundLevelPosition,...,sedimentSumpLength,x,y,geometry,ref_distance,dp,distance_to_ref,region,position,label
bro_identifier,,,,,,,,,,,,,,,,,,,,,
GMW000000082245-1,GMW000000082245,1,NEN5766,RDNAPTRANS2018,nee,POINT (63121.737 403079.799),51640813,WW,nee,5.89,...,0.5,63121.737,403079.799,POINT (63121.737 403079.799),81999.0,819.99,-1.910417,Os,broloket,GMW82245-1
GMW000000082245-2,GMW000000082245,2,NEN5766,RDNAPTRANS2018,nee,POINT (63121.737 403079.799),51640813,WW,nee,5.89,...,1.0,63121.737,403079.799,POINT (63121.737 403079.799),81999.0,819.99,-1.910417,Os,broloket,GMW82245-2
GMW000000082255-1,GMW000000082255,1,NEN5766,RDNAPTRANS2018,nee,POINT (63123.327 403066.59),51640813,WW,nee,5.83,...,1.0,63123.327,403066.590,POINT (63123.327 403066.59),81999.0,819.99,-15.211451,Os,broloket,GMW82255-1
GMW000000082255-2,GMW000000082255,2,NEN5766,RDNAPTRANS2018,nee,POINT (63123.327 403066.59),51640813,WW,nee,5.83,...,1.0,63123.327,403066.590,POINT (63123.327 403066.59),81999.0,819.99,-15.211451,Os,broloket,GMW82255-2


In [18]:
df_gmws.dp.unique()

array([ 784.37,  784.43, 1081.5 , 1081.51,  180.01,  157.99,  175.01,
        179.99,  151.98,  174.95,  145.99,  171.02,  171.  ,  140.01,
        152.01,  398.  ,  395.  ,  401.01,  400.94,  206.28,  206.27,
        206.72,  205.85,  207.  ,  789.02,  796.99,  793.  ,  792.99,
        789.01,  931.07, 1020.  ,  868.75,  893.95,  997.  ,  869.56,
        882.04,  894.  , 1052.  ,  917.09,  866.04, 1012.  ,  868.67,
       1029.  ,  930.5 ,  819.99,  853.98, 1076.01,  882.06,  866.03,
        869.93,  853.99,  917.06,  917.07,  970.01,  987.93,  975.  ,
        987.99,  980.  ,  969.99, 1502.67, 1503.  , 1503.05, 1504.17,
       1504.08, 1503.54, 1501.36, 1505.79, 1506.  , 1501.01, 1460.09,
       1461.01,  955.02,  783.11,  783.02,  971.05,  135.33, 1080.12,
         43.86, 1020.19, 1010.  ,  961.  ,  969.95,  982.99, 1000.02,
         43.88,  982.79,  367.76,  777.51, 1025.  ,  367.04,  963.99,
        995.79, 1005.01,  940.99,  367.89,  995.99,  135.52,  781.65,
        777.5 , 1067

Section that includes 2 hectometers (dp) behind and ahead.

In [19]:
if 0:
    fig, fn = gl.create_figure_for_dp(
        820,
        df_section, 
        geoprofile_cols, 
        df_profiles,
        oc_gwl,
        delta_dp=2,
        plot_path=PLOT_PATH,
        )